In [ ]:
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import tensorflow as tf

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/robinreni/signature-verification-dataset/code")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: hghff
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/robinreni/signature-verification-dataset


100%|██████████| 601M/601M [00:32<00:00, 19.2MB/s]


In [ ]:
df=pd.read_csv("/content/signature-verification-dataset/sign_data/train_data.csv")

In [ ]:
df.head()

,068/09_068.png,068_forg/03_0113068.PNG,1
0,068/09_068.png,068_forg/01_0124068.PNG,1
1,068/09_068.png,068_forg/02_0124068.PNG,1
2,068/09_068.png,068_forg/01_0113068.PNG,1
3,068/09_068.png,068_forg/04_0124068.PNG,1
4,068/09_068.png,068_forg/04_0113068.PNG,1


In [ ]:
df['1'][3]

1

In [ ]:
path = '/content/signature-verification-dataset/sign_data/train//'

In [ ]:
x = []
y = []
for ind in tqdm(df.index):
    name1 = df['068/09_068.png'][ind]
    name2 = df['068_forg/03_0113068.PNG'][ind]

    import os
    img1_path = os.path.join(path, name1)
    img2_path = os.path.join(path, name2)

    img1 = cv2.cvtColor(cv2.imread(img1_path), cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(cv2.imread(img2_path), cv2.COLOR_BGR2GRAY)

    img1 = np.array(img1).astype('float32')/255
    img2 = np.array(img2).astype('float32')/255
    img1 = cv2.resize(img1, (100,100), cv2.INTER_CUBIC)
    img2 = cv2.resize(img2, (100,100), cv2.INTER_CUBIC)

    # Reshape to add channel dimension
    img1 = img1.reshape(100, 100, 1)
    img2 = img2.reshape(100, 100, 1)

    x += [[img1, img2]]
    y += [df['1'][ind]]

x = np.array(x)
y = np.array(y)

100%|██████████| 23205/23205 [02:22<00:00, 163.16it/s]


In [ ]:
x_train1 = x[:17000,0]
x_train2 = x[:17000,1]

In [ ]:
x_train2.shape

(17000, 100, 100, 1)

In [ ]:
from tensorflow.keras import Input
from tensorflow.keras import layers
from tensorflow.keras import regularizers

In [ ]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    return tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))

In [ ]:
input = layers.Input((100, 100, 1))
x = tf.keras.layers.BatchNormalization()(input)
x = layers.Conv2D(32, (5, 5), activation="relu",kernel_regularizer=regularizers.L2(l2=2e-4),
    bias_regularizer=regularizers.L2(2e-4))(x)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(32, (5, 5),kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
    bias_regularizer=regularizers.L2(2e-4), activation="relu")(x)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Dropout(0.3)(x)
x = layers.Flatten()(x)


x = tf.keras.layers.BatchNormalization()(x)
x = layers.Dense(100, activation="relu")(x)
embedding_network = tf.keras.Model(input, x)


input_1 = layers.Input((100, 100, 1))
input_2 = layers.Input((100, 100, 1))

# As mentioned above, Siamese Network share weights between
# tower networks (sister networks). To allow this, we will use
# same embedding network for both tower networks.
tower_1 = embedding_network(input_1)
tower_2 = embedding_network(input_2)

merge_layer = layers.Lambda(euclidean_distance)([tower_1, tower_2])
normal_layer = tf.keras.layers.BatchNormalization()(merge_layer)
output_layer = layers.Dense(1, activation="sigmoid")(normal_layer)
siamese = tf.keras.Model(inputs=[input_1, input_2], outputs=output_layer)

In [ ]:
siamese.compile(loss='binary_crossentropy', optimizer="adam", metrics=["accuracy"])
siamese.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 100, 100, 1)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_2             │ (None, 100, 100, 1)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional (Functional)   │ (None, 100)            │      1,637,320 │ input_layer_1[0][0],   │
│                           │                        │                │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda (Lambda)           │ (None, 1)              │              0 │ functional[0][0],      │
│                           │                        │                │ functional[1][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 1)              │              4 │ lambda[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 1)              │              2 │ batch_normalization_2… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,637,326 (6.25 MB)

 Trainable params: 1,606,346 (6.13 MB)

 Non-trainable params: 30,980 (121.02 KB)

In [ ]:
siamese.fit(
    [x_train1, x_train2],
    y,
    validation_split = 0.1,
    batch_size = 32,
    epochs = 1,
    verbose = 1
)

479/479 ━━━━━━━━━━━━━━━━━━━━ 24s 28ms/step - accuracy: 0.8854 - loss: 0.4681 - val_accuracy: 0.8159 - val_loss: 0.4470


In [ ]:
siamese.save('siamese_model.h5')